In [73]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from src.cort_processor import *

from src.plotter import *
from src.filters import *
import math
import pickle
import scipy
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import cm
from scipy import signal
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from matplotlib import colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import tdt 
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, iirnotch, filtfilt, resample, hilbert, welch, resample
from scipy.fftpack import fft,fftfreq,rfft,irfft,ifft
from scipy.io import savemat
from src.phase_decoder_support import *
from src.neural_analysis import *
from src.wiener_filter import *
from src.folder_handler import *
from src.tdt_support import *
from src.decoders import *
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.datasets import make_classification

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
meepo1004 = CortProcessor('/mnt/c/oobootoo/rat-fes/data/1004_test')
meepo1004.process(clear_storage=False)

read from t=0s to t=246.43s
read from t=0s to t=298.5s
gib bins: 4083
gib bins: 5404


(array([[  0.,  20.,  20., ...,   0.,   0.,  20.],
        [ 60.,  20.,  60., ...,  20.,  20.,  20.],
        [ 20.,   0.,   0., ...,  40.,  20.,  40.],
        ...,
        [ 40.,   0.,  20., ..., 160.,  20.,  40.],
        [ 40.,  40.,  60., ...,   0.,  40.,  60.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]),
 array([[ 96.83447267,  30.16114856,  32.36723442, ...,  67.29106944,
          75.28234185, 146.50262126],
        [112.59527659,  55.01824894,  49.48172506, ...,  75.61167036,
          74.86562375, 137.055587  ],
        [106.81237871,  49.07522976,  41.48453291, ...,  73.9578064 ,
          71.42929072, 132.80626531],
        ...,
        [112.81929567,  54.76573498,  88.10683413, ...,  87.56055789,
          82.85515475, 132.20297142],
        [112.92536476,  59.93860243,  93.82667792, ...,  86.41129334,
          81.0329705 , 129.83160599],
        [131.70981432,  47.65634693, 100.78257682, ...,  92.90981426,
          80.21898788, 120.67302019]]))

In [3]:
phase_data2 = pd.read_csv('/mnt/c/oobootoo/rat-fes/data/1004_test/phase/1004_154010_phase.csv', header=0)
phase_data1 = pd.read_csv('/mnt/c/oobootoo/rat-fes/data/1004_test/phase/1004_153140_phase.csv', header=0)

In [4]:
time00 = phase_data1[phase_data1['fnum']==meepo1004.kin_data[0]['fnum'][0]].index.values
time01 = phase_data1[phase_data1['fnum']==meepo1004.kin_data[0]['fnum'][-1]].index.values
time10 = phase_data2[phase_data2['fnum']==meepo1004.kin_data[1]['fnum'][0]].index.values
time11 = phase_data2[phase_data2['fnum']==meepo1004.kin_data[1]['fnum'][-1]].index.values

In [29]:
meepo1004.kin_data[0]['fnum'][0]

3800

In [6]:
meepo1004.kin_data[1]['fnum'][-1]

55999

In [7]:
print(time00, time01,time10, time11)

[498] [41297] [949] [54948]


In [23]:
meepo1004.data["rates"][0].shape

(4083, 32)

In [42]:
phase1 = np.array(phase_data1.iloc[time00[0]:time01[0],-1:])
print(phase1)
phase2 = np.array(phase_data2.iloc[time10[0]:time11[0],-1:])
print(phase2)

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [27]:
resampled_phase1 = (np.rint(resample(phase1, meepo1004.data["rates"][0].shape[0], axis=0))).astype(int)
resampled_phase2 = (np.rint(resample(phase2, meepo1004.data["rates"][1].shape[0], axis=0))).astype(int)


In [45]:
binned_phase = []
binned_phase.append(resampled_phase1)
binned_phase.append(resampled_phase2)

print(binned_phase[0].shape)
print(meepo1004.data["rates"][0].shape)
print(len(binned_phase))
print(len(meepo1004.data["rates"]))

(4083, 1)
(4083, 32)
2
2


In [56]:
formatted_rates, formatted_phase = meepo1004.stitch_and_format(meepo1004.data["rates"],binned_phase)

In [76]:
X = formatted_rates

In [77]:
y = formatted_phase

In [79]:

# define dataset
# X, y = make_classification(n_samples=9467, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# # summarize the dataset
# print(X.shape, y.shape)
print((Counter(y)[2]+Counter(y)[1]+Counter(y)[0])/Counter(y)[0])
print((Counter(y)[2]+Counter(y)[1]+Counter(y)[0])/Counter(y)[1])
print((Counter(y)[2]+Counter(y)[1]+Counter(y)[0])/Counter(y)[2])
print(Counter(y)[2])

3.4053956834532375
1.864683868426236
5.880124223602484
1610


In [80]:
weights = {0:y.shape[0]/Counter(y)[0],1:y.shape[0]/Counter(y)[1],2:y.shape[0]/Counter(y)[2]} 
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', class_weight=weights)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [83]:
print(np.mean(scores))

0.48357522326727403
